# Lede Summer 2019 Project - Part 2
# Make a database of female and male Olympic all-time medallists (summer)
* Third attempt, use just 'most medals won' and scrape for medal count

In [1]:
import requests
import pandas as pd
import re
import numpy as np
import os

import itertools

from bs4 import BeautifulSoup
from selenium import webdriver

#### LINKS:
https://www.olympic.org/factsheets-and-reference-documents/records-and-medals

WINTER:
https://stillmed.olympic.org/media/Document%20Library/OlympicOrg/Factsheets-Reference-Documents/Games/Records-and-Medals/Factsheet-Records-of-medals-at-the-Olympic-Winter-Games.pdf#_ga=2.127809805.1432842169.1561491999-1688621044.1561491999

### Turn pdf to text

In [3]:
cols = ['gender', 'medal_count', 'last_name', 'first_name', 'citizenship', 'event', 'years']
df = pd.read_csv('oly_summer_most.txt', sep=',', names=cols, index_col=False)

### Manually clean the oly_summer_most.txt file using text editor

* Standardise format of rows by adding columns, replacing dashes with commas, eliminating white space, formatting years so that they follow \d\d\d\d-\d\d\d\d format

### Fill in values for medal type and gender

In [4]:
df['gender'] = df.gender.fillna(method='ffill')
df = df.dropna(subset = ['last_name'])

In [5]:
# df['medal_type'] = df.medal_type.str.extract('Most (\w+) medals.*')
df['gender'] = df.gender.str.extract('.*[(](\w+)[)]$')
df.shape

(29, 7)

### Make a new column with the full name

In [6]:
df['full_name'] = (df.first_name+df.last_name).str.strip()

# Scrape medals data for athletes with missing data
* Make endpoints for urls
* Then clean up the endpoint for the extra dashes, Jr.s
* Make a list of endpoints over which I can interate
* Clean up the list of endpoints for athletes with a middle name or a hyphenated last names

In [7]:
df['endpoint'] = df.full_name.str.replace(' ','-')

In [8]:
# Clean up endpoint
df['endpoint'] = df.endpoint.str.replace('--','-')
df['endpoint'] = df.endpoint.str.replace('.','')
# df

In [9]:
# Make a list of endpoints
athletes = df.endpoint.to_list()
len(athletes)

29

In [10]:
## Clean the list for athletes with middle names and hyphenated last names

for idx, name in enumerate(athletes):
    if name == 'Carl-Townsend-OSBURN':
        athletes[idx] = 'Carl-OSBURN'
    elif name == 'Birgit-FISCHER-SCHMIDT':
        athletes[idx] = 'Birgit-FISCHER'
    else:
        pass
    
len(athletes)

29

### Edit the endpoints in the original dataframe so that they can be merged on endpoints later on!!!

In [11]:
df['endpoint_cleaned'] = athletes

In [12]:
rows = []
for athlete in athletes:
    try:
        url = f'https://www.olympic.org/{athlete}'
        response = requests.get(url)
        doc = BeautifulSoup(response.text)
        medals = doc.find(class_='medal-box').text.strip().split( ' \n\n\n ')
        
        row = {}

        gold = int(medals[0][1:])
        silver = int(medals[1][1:])
        bronze = int(medals[2][1:])
        
        row['endpoint'] = athlete
        row['medals_gold'] = gold
        row['medals_silver'] = silver
        row['medals_bronze'] = bronze
        row['medals_total'] = gold + silver + bronze
    except:
        print('-----')
        print('ERROR WITH', athlete)
        print('-----')
    rows.append(row)

In [13]:
len(rows)

29

In [14]:
df_medals = pd.DataFrame(rows)
df_medals.shape

(29, 5)

In [15]:
merged = df.merge(df_medals, left_on='endpoint_cleaned', right_on='endpoint')

In [16]:
merged.shape

(29, 15)

In [17]:
# Check to see the total medal counts match up
merged[merged.medal_count != merged.medals_total]

,gender,medal_count,last_name,first_name,citizenship,event,years,full_name,endpoint_x,endpoint_cleaned,endpoint_y,medals_bronze,medals_gold,medals_silver,medals_total
3,men,13.0,MANGIAROTTI,Edoardo,ITA,fencing,1936-1960,Edoardo MANGIAROTTI,Edoardo-MANGIAROTTI,Edoardo-MANGIAROTTI,Edoardo-MANGIAROTTI,2,5,5,12
16,men,10.0,GEREVICH,Aladar,HUN,fencing,1932-1960,Aladar GEREVICH,Aladar-GEREVICH,Aladar-GEREVICH,Aladar-GEREVICH,2,5,1,8


# FACT CHECK

The medal count for MANGIAROTTI should be 13. (13 medals - 6 gold, 5 silver, 2 bronze)
* http://www.espn.com/olympics/summer08/fanguide/athlete?athlete=8612

The medal count for GEREVICH should be 10. (7 Gold, 1 Silver, 2 Bronze)
* https://www.sports-reference.com/olympics/athletes/ge/aladar-ger

## Manually edit incorrect values 
* MANGIAROTTI should be 13. (13 medals - 6 gold, 5 silver, 2 bronze), and

* GEREVICH should be 10. (7 Gold, 1 Silver, 2 Bronze)

In [18]:
rows[3]['medals_gold'] = 6
rows[3]['medals_total'] = 13
rows[3]

{'endpoint': 'Edoardo-MANGIAROTTI',
 'medals_gold': 6,
 'medals_silver': 5,
 'medals_bronze': 2,
 'medals_total': 13}

In [19]:
rows[16]['medals_gold'] = 7
rows[16]['medals_total'] = 10
rows[16]

{'endpoint': 'Aladar-GEREVICH',
 'medals_gold': 7,
 'medals_silver': 1,
 'medals_bronze': 2,
 'medals_total': 10}

# Re-do the dataframe transformation so that the correct medal counts are in the final, 'merged' dataframe

In [20]:
df_medals = pd.DataFrame(rows)
df_medals.shape

(29, 5)

In [21]:
merged = df.merge(df_medals, left_on='endpoint_cleaned', right_on='endpoint')

In [22]:
merged.shape

(29, 15)

In [23]:
# Check to see the total medal counts match up
merged[merged.medal_count != merged.medals_total]

,gender,medal_count,last_name,first_name,citizenship,event,years,full_name,endpoint_x,endpoint_cleaned,endpoint_y,medals_bronze,medals_gold,medals_silver,medals_total


## Now, medal_count and medals_total match.
* Remove medal_count, endpoint_x, endpoint_y columns to avoid confusion

In [24]:
merged = merged.drop(columns=["medal_count", 'endpoint_x', 'endpoint_y'])

In [25]:
merged['gender'] = merged.gender.str.title()

In [26]:
merged['game_type'] = 'Olympic'
merged['season'] = 'Summer'

## Add the other_info and alternate_name columns so that the columns match Paralympic dataframes

In [27]:
merged['other_info'] = ''
merged['alternate_name'] = ''

In [28]:
merged

,gender,last_name,first_name,citizenship,event,years,full_name,endpoint_cleaned,medals_bronze,medals_gold,medals_silver,medals_total,game_type,season,other_info,alternate_name
0,Men,PHELPS,Michael,USA,aquatics,2004-2016,Michael PHELPS,Michael-PHELPS,2,23,3,28,Olympic,Summer,,
1,Men,ANDRIANOV,Nikolay,URS,gymnastics,1972-1980,Nikolay ANDRIANOV,Nikolay-ANDRIANOV,3,7,5,15,Olympic,Summer,,
2,Men,SHAKHLIN,Boris,URS,gymnastics,1956-1964,Boris SHAKHLIN,Boris-SHAKHLIN,2,7,4,13,Olympic,Summer,,
3,Men,MANGIAROTTI,Edoardo,ITA,fencing,1936-1960,Edoardo MANGIAROTTI,Edoardo-MANGIAROTTI,2,6,5,13,Olympic,Summer,,
4,Men,ONO,Takashi,JPN,gymnastics,1952-1964,Takashi ONO,Takashi-ONO,4,5,4,13,Olympic,Summer,,
5,Men,KATO,Sawao,JPN,gymnastics,1968-1976,Sawao KATO,Sawao-KATO,1,8,3,12,Olympic,Summer,,
6,Men,NURMI,Paavo,FIN,athletics,1920-1928,Paavo NURMI,Paavo-NURMI,0,9,3,12,Olympic,Summer,,
7,Men,NEMOV,Alexei,RUS,gymnastics,1996-2000,Alexei NEMOV,Alexei-NEMOV,6,4,2,12,Olympic,Summer,,
8,Men,LOCHTE,Ryan,USA,aquatics,2004-2016,Ryan LOCHTE,Ryan-LOCHTE,3,6,3,12,Olympic,Summer,,
9,Men,OSBURN,Carl Townsend,USA,shooting,1912-1924,Carl Townsend OSBURN,Carl-OSBURN,2,5,4,11,Olympic,Summer,,


### Check for NaN values

In [29]:
merged.isna().sum()

gender              0
last_name           0
first_name          0
citizenship         0
event               0
years               0
full_name           0
endpoint_cleaned    0
medals_bronze       0
medals_gold         0
medals_silver       0
medals_total        0
game_type           0
season              0
other_info          0
alternate_name      0
dtype: int64

In [30]:
merged.to_csv('oly_summer_complete.csv', index=False)